In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataframe = pd.read_csv("Google Finance.csv")
dataframe.head()

,Date,Close
0,02/01/2023 16:56:00,"14,3"
1,03/01/2023 16:56:00,"14,57"
2,04/01/2023 16:56:00,"14,97"
3,05/01/2023 16:56:00,"14,8"
4,06/01/2023 16:56:00,"14,55"


# Tratamento de Dados

## Filtros

Os filtros a seguir não se aplicam ao projeto atual devido ao dataset utilizado, porém podem ser interessantes posteriormente.

In [3]:
# Filtrar colunas com mais de 90% de elementos nulos

# dataframe = dataframe[dataframe.columns[dataframe.isnull().mean() < 0.90]]

In [4]:
# Tratando NaN's

# dataframe = dataframe.interpolate()
# dataframe = dataframe.fillna(method='bfill')

# Tratamento para a Rede Neural

## Criando entradas e saídas de treino

As entradas (X_train) são os 60 valores anteriores do preço da ação. A saída é o valor seguinte.

In [6]:
training_set = dataframe.iloc[:, 1:2].values

X_train = []
y_train = []

for i in range(60, len(training_set)):
    X_train.append(training_set[i-60:i, 0])
    y_train.append(training_set[i, 0])